# Speed and precision benchmark between polars vs pandas in reading csv

In [ ]:
############################################################
#put your code for reading the csv data here
############################################################
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/

import pandas as pd
import polars as pl

import torch.utils.benchmark as benchmark

def pandas_read_csv(csv_train, csv_val, csv_test):
    """
    Args:
        csv_train (str): name of train set csv file
        csv_val (str): name of val set csv file
        csv_test (str): name of test set csv file

    Output:
        read train, val, test sets in DataFrame format
    """
    df_train = pd.read_csv(csv_train)
    df_val = pd.read_csv(csv_val)
    df_test = pd.read_csv(csv_test)

    # uncomment this to verify floating value difference
    #df_train.to_csv("df_train.csv", index=False)
    #df_val.to_csv("df_val.csv", index=False)
    #df_test.to_csv("df_test.csv", index=False)

    # uncomment return when not doing speed benchmark
    return df_train, df_val, df_test

def polar_read_csv(csv_train, csv_val, csv_test):
    """
    Args:
        csv_train (str): name of train set csv file
        csv_val (str): name of val set csv file
        csv_test (str): name of test set csv file

    Output:
        read train, val, test sets in DataFrame format
    """

    df_train = pl.read_csv(csv_train)
    df_val = pl.read_csv(csv_val)
    df_test = pl.read_csv(csv_test)

    # uncomment this to verify floating value difference
    #polar_train.write_csv("polar_train.csv")
    #polar_val.write_csv("polar_val.csv")
    #polar_test.write_csv("polar_test.csv")

    # uncomment return when not doing speed benchmark
    return df_train, df_val, df_test

pandas_read_csv("UNSWNB15_training_coursework.csv", "UNSWNB15_testing1_coursework.csv", "UNSWNB15_testing2_coursework_no_label.csv")
polar_read_csv("UNSWNB15_training_coursework.csv", "UNSWNB15_testing1_coursework.csv", "UNSWNB15_testing2_coursework_no_label.csv")

Mounted at /content/drive
/content/drive/MyDrive


No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


(shape: (20_000, 44)
 ┌───────┬──────────┬───────┬─────────┬───┬────────────┬────────────┬─────────────────┬───────┐
 │ id    ┆ dur      ┆ proto ┆ service ┆ … ┆ ct_src_ltm ┆ ct_srv_dst ┆ is_sm_ips_ports ┆ label │
 │ ---   ┆ ---      ┆ ---   ┆ ---     ┆   ┆ ---        ┆ ---        ┆ ---             ┆ ---   │
 │ i64   ┆ f64      ┆ str   ┆ str     ┆   ┆ i64        ┆ i64        ┆ i64             ┆ i64   │
 ╞═══════╪══════════╪═══════╪═════════╪═══╪════════════╪════════════╪═════════════════╪═══════╡
 │ 1     ┆ 0.000003 ┆ unas  ┆ -       ┆ … ┆ 8          ┆ 11         ┆ 0               ┆ 1     │
 │ 2     ┆ 0.885807 ┆ tcp   ┆ ftp     ┆ … ┆ 5          ┆ 1          ┆ 0               ┆ 0     │
 │ 3     ┆ 0.538781 ┆ tcp   ┆ http    ┆ … ┆ 2          ┆ 6          ┆ 0               ┆ 0     │
 │ 4     ┆ 0.000008 ┆ udp   ┆ dns     ┆ … ┆ 27         ┆ 34         ┆ 0               ┆ 1     │
 │ 5     ┆ 0.448734 ┆ tcp   ┆ ftp     ┆ … ┆ 1          ┆ 1          ┆ 0               ┆ 1     │
 │ …     ┆ …       

In [ ]:
# uncommment this for speed benchmark
t0 = benchmark.Timer(
    stmt='pandas_read_csv(csv_train, csv_val, csv_test)',
    setup='from __main__ import pandas_read_csv',
    globals={'csv_train': "UNSWNB15_training_coursework.csv", "csv_val": "UNSWNB15_testing1_coursework.csv", "csv_test": "UNSWNB15_testing2_coursework_no_label.csv"})

t1 = benchmark.Timer(
    stmt='polar_read_csv(csv_train, csv_val, csv_test)',
    setup='from __main__ import polar_read_csv',
    globals={'csv_train': "UNSWNB15_training_coursework.csv", "csv_val": "UNSWNB15_testing1_coursework.csv", "csv_test": "UNSWNB15_testing2_coursework_no_label.csv"})

print(t0.timeit(1000))
print(t1.timeit(1000))

pandas_read_csv(csv_train, csv_val, csv_test)
setup: from __main__ import pandas_read_csv
  173.19 ms
  1 measurement, 1000 runs , 1 thread
polar_read_csv(csv_train, csv_val, csv_test)
setup: from __main__ import polar_read_csv
  60.18 ms
  1 measurement, 1000 runs , 1 thread


### Precision benchmark for Data-Processing

In [ ]:
import numpy as np

a = np.float32(1)
b = np.float32(2)
c1 = np.float32(266666656)
c2 = np.float32(266661656)
print(f"{b/c1:.30f}")
print(f"{b/c2:.30f}")
print(f"{a/c1:.30f}")
print(f"{a/c2:.30f}")

0.000000007500000620552782493178
0.000000007500140952743095112965
0.000000003750000310276391246589
0.000000003750070476371547556482


In [ ]:
import numpy as np
import polars as pl
import json

def encode_category_attribute(csv_train, csv_val, csv_test, attribute_list, dict_output_file="train_category.json"):
    df_train, df_val, df_test = polar_read_csv(csv_train, csv_val, csv_test)

    attribute_dict = {}
    for attribute in attribute_list:
        if attribute in df_train.columns:
            unique_vals = (df_train.select(attribute).drop_nulls().unique().to_series().to_list())
            attribute_dict[attribute] = {val: idx for idx, val in enumerate(unique_vals)}
        else:
            print(f"Warning: '{attribute}' not found in training CSV.")

    def apply_encoding(df, attribute_dict):
        for attribute, val_to_idx in attribute_dict.items():
            if attribute in df.columns:
                df = df.with_columns(pl.col(attribute).replace(val_to_idx).alias(attribute))
        return df
    df_train_encoded = apply_encoding(df_train, attribute_dict)
    df_val_encoded = apply_encoding(df_val, attribute_dict)
    df_test_encoded = apply_encoding(df_test, attribute_dict)

    # save encoded csv and .json to verify difference between using pandas vs polars to encode csv file
    with open(dict_output_file, 'w') as f:
        json.dump(attribute_dict, f, indent=4)
    #df_train_encoded.write_csv("polar_train_encoded.csv")
    #df_val_encoded.write_csv("polar_val_encoded.csv")
    #df_test_encoded.write_csv("polar_test_encoded.csv")

    return df_train_encoded, df_val_encoded, df_test_encoded


def get_mean_std_after_rescaled(x: np.ndarray):
    scale = np.max(x)
    x_rescaled = x/scale
    mean = np.mean(x_rescaled, axis=0, keepdims=True)
    std = np.std(x_rescaled, axis=0, keepdims=True)
    return mean, std

def fused_rescale_normalise(x: np.ndarray, mean, std):
    scaling_factor = np.max(x)
    return (x - mean*scaling_factor) / (std*scaling_factor)

def rescale_then_normalise(x: np.ndarray):
    scale = np.max(x)
    x_rescaled = x/scale
    mean = np.mean(x_rescaled, axis=0)
    std = np.std(x_rescaled, axis=0)
    return (x_rescaled - mean) / std

train_df_encoded, val_df_encoded, test_df_encoded = encode_category_attribute("UNSWNB15_training_coursework.csv",
                                                                              "UNSWNB15_testing1_coursework.csv",
                                                                              "UNSWNB15_testing2_coursework_no_label.csv",
                                                                               ["proto", "service", "state"])
val_df_encoded = val_df_encoded.filter(pl.col("state") != "RST")

train_arr = train_df_encoded.to_numpy().astype(np.float32)
val_arr = val_df_encoded.to_numpy().astype(np.float32)
test_arr = test_df_encoded.to_numpy().astype(np.float32)

train_features = train_arr[:, 1:-1]
val_features = val_arr[:, 1:-1]
test_features = test_arr[:, 1:]

mean, std = get_mean_std_after_rescaled(train_features)
train_set = np.concatenate([fused_rescale_normalise(train_features, mean, std), train_arr[:, -1].reshape(-1, 1)], axis=1)
val_set = np.concatenate([fused_rescale_normalise(val_features, mean, std), val_arr[:, -1].reshape(-1, 1)], axis=1)
test_set = fused_rescale_normalise(test_features, mean, std)

separate_train_set = np.concatenate([rescale_then_normalise(train_features), train_arr[:, -1].reshape(-1, 1)], axis=1)
print("Max absolute difference between fused scale-normalise and scale-then-normalise", np.max(np.abs(train_set - separate_train_set)))

print(train_set.shape)
num_features = train_set.shape[1]
column_names = [f"feature_{i+1}" for i in range(num_features)]

df = pl.DataFrame(train_set)
df.write_csv("normalised_train_features.csv")

Max absolute difference between fused scale-normalise and scale-then-normalise 3.8146973e-06
(20000, 43)


In [ ]:
batch_size = 16
batches = [train_set[i:i+batch_size] for i in range(0, train_set.shape[0], batch_size)]
batch_0 = batches[0]
labels_0 = batch_0[:, -1].reshape(-1, 1)
features_0 = np.expand_dims(batch_0[:, :-1], axis=-1)
print(features_0.shape)

(16, 42, 1)


# Precision benchmark for model implementation

### Linear layer

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import math

# ---------- Kaiming uniform distribution to init weight & bias of Linear layer ----------
def calculate_gain(nonlinearity, param=None):
    # Inspired from https://github.com/pytorch/pytorch/blob/main/torch/nn/init.py#L72
    if nonlinearity in ['linear', 'conv1d', 'conv2d', 'conv3d',
                        'conv_transpose1d', 'conv_transpose2d', 'conv_transpose3d', 'sigmoid']:
        return 1.0
    elif nonlinearity == 'tanh':
        return 5.0 / 3
    elif nonlinearity == 'relu':
        return math.sqrt(2.0)
    elif nonlinearity == 'leaky_relu':
        negative_slope = 0.01 if param is None else param
        return math.sqrt(2.0 / (1 + negative_slope ** 2))
    else:
        raise ValueError("Unsupported nonlinearity {}".format(nonlinearity))


def _calculate_correct_fan(tensor, mode):
    # Inspired from _calculate_fan_in_and_fan_out: https://github.com/pytorch/pytorch/blob/main/torch/nn/init.py#L345
    # Note that pytorch uses _calculate_correct_fan as API for _calculate_fan_in_and_fan_out so I merely remove the API
    if tensor.ndim < 2:
        raise ValueError("Fan in and fan out cannot be computed for tensor with fewer than 2 dimensions")
    if mode == "fan_in":
        num_input_fmaps = tensor.shape[1]
        receptive_field_size = np.prod(tensor.shape[2:]) if tensor.ndim > 2 else 1
        return num_input_fmaps * receptive_field_size
    elif mode == "fan_out":
        num_output_fmaps = tensor.shape[0]
        receptive_field_size = np.prod(tensor.shape[2:]) if tensor.ndim > 2 else 1
        return num_output_fmaps * receptive_field_size
    else:
        raise ValueError("mode must be either 'fan_in' or 'fan_out'")


def kaiming_uniform_(tensor, a=0, mode="fan_in", nonlinearity="leaky_relu"):
    # Inspired from https://github.com/pytorch/pytorch/blob/main/torch/nn/init.py#L456
    fan = _calculate_correct_fan(tensor, mode)
    gain = calculate_gain(nonlinearity, a)
    std = gain / math.sqrt(fan)
    bound = math.sqrt(3.0) * std
    tensor[:] = np.random.uniform(-bound, bound, tensor.shape).astype(np.float32)
    return tensor


class Linear:
    # Inspired from https://github.com/pytorch/pytorch/blob/main/torch/nn/modules/linear.py#L50
    def __init__(self, in_features, out_features, bias=True):
        self.in_features = in_features
        self.out_features = out_features
        self.weight = np.empty((out_features, in_features), dtype=np.float32)
        self.bias = np.empty((out_features,), dtype=np.float32) if bias else None
        self.reset_parameters()

    def reset_parameters(self):
        kaiming_uniform_(self.weight, a=math.sqrt(5), mode="fan_in", nonlinearity="leaky_relu")
        if self.bias is not None:
            fan_in = _calculate_correct_fan(self.weight, "fan_in")
            bound = 1 / math.sqrt(fan_in)
            self.bias[:] = np.random.uniform(-bound, bound, self.bias.shape).astype(np.float32)

    def forward(self, input):
        """
        Args:
            input (np.ndarray): shape (batch_size, N, in_features)

        Returns:
            np.ndarray: shape (batch_size, N, out_features)
        """

        #output = np.einsum('bni,oi->bno', input, self.weight)
        #if self.bias is not None:
        #    output += self.bias.reshape(1, 1, -1)
        #return output

        # Original implementation # Observed no difference between einsum and original implementation
        return np.matmul(np.ascontiguousarray(input), self.weight.T) + (self.bias if self.bias is not None else 0)

    def backward(self, grad_output, input):
        """
        Args:
            grad_output (np.ndarray): Gradient w.r.t. output with shape (B, N, out_features)
            input (np.ndarray): Original input with shape (B, N, in_features)

        Returns:
            grad_input (np.ndarray): Gradient w.r.t. input with shape (B, N, in_features)
            grad_weight (np.ndarray): Gradient w.r.t. weight with shape (out_features, in_features)
            grad_bias (np.ndarray or None): Gradient w.r.t. bias with shape (out_features,)

        I find floating difference between torch.nn.Linear and this code is smaller when I use Einsum implementation of grad_weight
        """
        grad_input = np.matmul(grad_output, self.weight)

        # Matmul implementation of grad_weight:
        #input_reshaped = input.reshape(B * N, self.in_features)
        #grad_output_reshaped = grad_output.reshape(B * N, self.out_features)
        #grad_weight = np.matmul(grad_output_reshaped.T, input_reshaped)

        # Einsum implementation of grad_weight:
        if input.ndim == 2:
            self.grad_weight = np.einsum('bi,bj->ij', grad_output, input)
            self.grad_bias = np.sum(grad_output, axis=0) if self.bias is not None else None
        elif input.ndim == 3:
            self.grad_weight = np.einsum('bnk,bnl->kl', grad_output, input)
            self.grad_bias = np.sum(grad_output, axis=(0, 1))

        return grad_input, self.grad_weight, self.grad_bias

# ---------- Comparison with torch.nn.Linear ----------
B, N, C = features_0.shape
x_torch = torch.tensor(features_0, dtype=torch.float32, requires_grad=True)
embed_dim = 128

custom_linear = Linear(C, embed_dim, bias=True)

torch_linear = nn.Linear(C, embed_dim, bias=True)
with torch.no_grad():
    torch_linear.weight.copy_(torch.tensor(custom_linear.weight))
    if custom_linear.bias is not None:
        torch_linear.bias.copy_(torch.tensor(custom_linear.bias))

# ---------- Forward Pass Comparison ----------
output_np = custom_linear.forward(features_0)
output_torch = torch_linear(x_torch)
print("Forward pass comparison max abs difference:", np.abs(output_np - output_torch.detach().numpy()).max())

# ---------- Backward Pass Comparison ----------
# Dummy gradient in numpy
np.random.seed(1000)
grad_np = np.random.randn(B, N, embed_dim).astype(np.float32)
grad_torch = torch.tensor(grad_np, dtype=torch.float32)

# Manual backward pass in NumPy
grad_input_np, grad_weight_np, grad_bias_np = custom_linear.backward(grad_np, features_0)

# Autograd in PyTorch
output_torch.backward(grad_torch)
grad_input_torch = x_torch.grad.detach()
grad_weight_torch = torch_linear.weight.grad.detach()
grad_bias_torch = torch_linear.bias.grad.detach()

print("\nBackward pass comparison:")
print("Grad output max abs difference:", np.abs(grad_np - grad_torch.numpy()).max())
print("Grad input max abs difference:", np.abs(grad_input_np - grad_input_torch.numpy()).max())
print("Grad weight max abs difference:", np.abs(grad_weight_np - grad_weight_torch.numpy()).max())
print("Grad bias max abs difference:", np.abs(grad_bias_np - grad_bias_torch.numpy()).max())

Forward pass comparison max abs difference: 0.0

Backward pass comparison:
Grad output max abs difference: 0.0
Grad input max abs difference: 2.861023e-06
Grad weight max abs difference: 1.5258789e-05
Grad bias max abs difference: 3.8146973e-05


### GELU

In [ ]:
import numpy as np
import torch
import torch.nn.functional as F
from scipy.special import erf


class GELU:
    # Inspired from https://github.com/oniani/ai/blob/main/activation/gelu.py#L4
    def __init__(self):
        self.ctx = {}

    def forward(self, input):
        """
        Args:
            input (B, N, embed_dim)

        Returns
            GELU output of (B, N, embed_dim)
        """
        cdf = 0.5 * (1 + erf(input / np.sqrt(2)))
        self.ctx['input'] = input.copy()
        self.ctx['cdf'] = cdf.copy()
        return input * cdf

    def backward(self, grad_output):
        """
        Returns:
            Grad loss w.r.t to GELU input
        """
        input = self.ctx['input']
        cdf = self.ctx['cdf']
        pdf_val = 1 / np.sqrt(2 * np.pi) * np.exp(-0.5 * np.power(input, 2))
        grad_local = cdf + input * pdf_val

        # Reset ctx to prevent memory leak
        # self.ctx = {}
        return grad_output * grad_local


if __name__ == "__main__":
    np.random.seed(1000)
    embed_dim = 192

    # Compare forward pass
    x_np = np.random.randn(batch_size, N, embed_dim).astype(np.float32)
    gelu = GELU()
    y_np = gelu.forward(x_np)

    x_torch = torch.tensor(x_np, requires_grad=True)
    y_torch = F.gelu(x_torch)
    print("GELU max abs difference in forward pass:", np.abs(y_np - y_torch.detach().numpy()).max())

    # Compare backward pass:
    grad_np = np.random.randn(batch_size, N, embed_dim).astype(np.float32)
    grad_input_np = gelu.backward(grad_np)

    grad_torch = torch.tensor(grad_np, dtype=torch.float32)
    y_torch.backward(grad_torch)
    grad_input_torch = x_torch.grad.detach()

    # Compare the backward gradients (maximum absolute difference)
    print("GELU max abs difference in backward pass:", np.abs(grad_input_np - grad_input_torch.numpy()).max())


GELU max abs difference in forward pass: 3.541185931155155e-07
GELU max abs difference in backward pass: 9.027864429356214e-07


### Linear -> GELU

In [ ]:
class CustomModel:
    def __init__(self, in_features, out_features, bias=True):
        self.linear = Linear(in_features, out_features, bias=bias)
        self.gelu = GELU()

    def forward(self, input):
        self.linear_out = self.linear.forward(input)
        self.out = self.gelu.forward(self.linear_out)
        return self.out

    def backward(self, grad_output, input):
        grad_after_gelu = self.gelu.backward(grad_output)
        grad_input, grad_weight, grad_bias = self.linear.backward(grad_after_gelu, input)
        return grad_input, grad_weight, grad_bias

# ---------- Main Comparison ----------
if __name__ == "__main__":
    embed_dim = 128
    x_torch = torch.tensor(features_0, dtype=torch.float32, requires_grad=True)

    # ---------- Copy weight and bias from torch.nn.Linear -> numpy Linear ----------
    custom_model = CustomModel(C, embed_dim, bias=True)
    torch_linear = nn.Linear(C, embed_dim, bias=True)
    with torch.no_grad():
        torch_linear.weight.copy_(torch.tensor(custom_model.linear.weight))
        if custom_model.linear.bias is not None:
            torch_linear.bias.copy_(torch.tensor(custom_model.linear.bias))

    # ---------- Forward Pass Comparison ----------
    out_np = custom_model.forward(features_0)

    out_torch = F.gelu(torch_linear(x_torch))

    print("Combined model forward pass max abs difference:", np.abs(out_np - out_torch.detach().numpy()).max())

    # ---------- Backward Pass Comparison ----------
    np.random.seed(1000)
    grad_dummy_np = np.random.randn(B, N, embed_dim).astype(np.float32)
    grad_dummy_torch = torch.tensor(grad_dummy_np, dtype=torch.float32)

    # NumPy backward pass:
    grad_input_np, grad_weight_np, grad_bias_np = custom_model.backward(grad_dummy_np, features_0)

    # PyTorch backward pass: compute gradients via autograd
    out_torch.backward(grad_dummy_torch)
    grad_input_torch = x_torch.grad.detach()
    grad_weight_torch = torch_linear.weight.grad.detach()
    grad_bias_torch = torch_linear.bias.grad.detach()

    print("\nCombined model backward pass comparisons:")
    print("Grad input max abs difference:", np.abs(grad_input_np - grad_input_torch.numpy()).max())
    print("Grad weight max abs difference:", np.abs(grad_weight_np - grad_weight_torch.numpy()).max())
    print("Grad bias max abs difference:", np.abs(grad_bias_np - grad_bias_torch.numpy()).max())

Combined model forward pass max abs difference: 3.5736101633432327e-07

Combined model backward pass comparisons:
Grad input max abs difference: 1.986309933421637e-06
Grad weight max abs difference: 2.4437184062975348e-05
Grad bias max abs difference: 1.0478283925863252e-05


### LayerNorm

In [ ]:
import numpy as np
import torch
import torch.nn as nn

eps = 1e-5
class LayerNorm:
    def __init__(self, dim):
        self.dim = dim
        self.weight = np.ones((dim,), dtype=np.float32)
        self.bias = np.zeros((dim,), dtype=np.float32)

    def forward(self, x):
        """
        Args:
            x: (B, N, embed_dim)
        Returns:
            out: layer-normalized activations, shape (B, T, embed_dim)
            cache: tuple of (x, self.w, mean, rstd) for backward pass.
        """
        B, T, C = x.shape
        mean = np.sum(x, axis=-1, keepdims=True) / C           # (B, T, 1)
        xshift = x - mean
        var = np.sum(xshift ** 2, axis=-1, keepdims=True) / C  # (B, T, 1)
        rstd = (var + eps) ** (-0.5)                           # (B, T, 1)
        norm = xshift * rstd
        out = norm * self.weight + self.bias
        cache = (x, self.weight, mean, rstd)
        return out, cache

    def backward(self, dout, cache):
        """
        dout: upstream gradients, shape (B, N, embed_dim)
        cache: tuple of (x, w, mean, rstd) from forward pass.
        Returns:
            dx: gradient with respect to x, shape (B, T, embed_dim)
            dw: gradient with respect to self.w, shape (embed_dim,)
            db: gradient with respect to self.b, shape (embed_dim,)
        """
        x, w, mean, rstd = cache
        norm = (x - mean) * rstd
        db = np.sum(dout, axis=(0, 1))
        dw = np.sum(dout * norm, axis=(0, 1))
        dnorm = dout * w
        dnorm_mean = np.mean(dnorm, axis=-1, keepdims=True)
        dx = dnorm - dnorm_mean - norm * np.mean(dnorm * norm, axis=-1, keepdims=True)
        dx *= rstd
        self.grad_weight = dw
        self.grad_bias = db
        return dx, dw, db

if __name__ == "__main__":
    np.random.seed(42)
    shape = (128, 42, 192)
    x_np = np.random.randn(*shape).astype(np.float32)
    x_torch = torch.tensor(x_np, dtype=torch.float32, requires_grad=True)

    # Generate random weights and biases of shape (768,)
    C = shape[2]
    w_np = np.random.randn(C).astype(np.float32)
    b_np = np.random.randn(C).astype(np.float32)

    # Create PyTorch built-in LayerNorm and set its parameters.
    layernorm_torch = nn.LayerNorm(normalized_shape=C, eps=eps, elementwise_affine=True)
    layernorm_torch.weight.data = torch.tensor(w_np, dtype=torch.float32)
    layernorm_torch.bias.data = torch.tensor(b_np, dtype=torch.float32)

    # ---------------- Forward Pass Comparison ----------------
    # Create our NumPy LayerNorm and set its weight and bias to match
    layernorm_np = LayerNorm(C)
    layernorm_np.weight = w_np.copy()
    layernorm_np.bias = b_np.copy()
    out_np, cache = layernorm_np.forward(x_np)

    out_torch = layernorm_torch(x_torch)
    out_torch_np = out_torch.detach().numpy()

    print("Forward Pass Comparison:")
    print("Max abs difference:", np.max(np.abs(out_np - out_torch_np)))

    # ---------------- Backward Pass Comparison ----------------
    # Create random upstream gradients (dout) with the same shape
    dout_np = np.random.randn(*shape).astype(np.float32)
    dout_torch = torch.tensor(dout_np, dtype=torch.float32)

    # Backward pass using the NumPy implementation
    dx_np, dw_np, db_np = layernorm_np.backward(dout_np, cache)

    # Zero gradients before backward pass in PyTorch
    if x_torch.grad is not None:
        x_torch.grad.zero_()
    if layernorm_torch.weight.grad is not None:
        layernorm_torch.weight.grad.zero_()
    if layernorm_torch.bias.grad is not None:
        layernorm_torch.bias.grad.zero_()

    # Backward pass in PyTorch: supply dout_torch as gradient
    out_torch.backward(dout_torch)
    dx_torch = x_torch.grad.detach().numpy()
    dw_torch = layernorm_torch.weight.grad.detach().numpy()
    db_torch = layernorm_torch.bias.grad.detach().numpy()

    print("\nBackward Pass Comparison:")
    print("dx max abs difference:", np.max(np.abs(dx_np - dx_torch)))
    print("dw max abs difference:", np.max(np.abs(dw_np - dw_torch)))
    print("db max abs difference:", np.max(np.abs(db_np - db_torch)))


Forward Pass Comparison:
Max abs difference: 1.9073486e-06

Backward Pass Comparison:
dx max abs difference: 2.861023e-06
dw max abs difference: 7.8201294e-05
db max abs difference: 0.0


### DropOut

torch.manual_seed(42) != numpy.manual_seed(42) so to fairly compare, I need to copy dropout mask from pytorch to numpy. torch.nn.Dropout doesn't return mask so I create a children class of torch.nn.Dropout that stores mask

In [ ]:
import numpy as np
import torch
import torch.nn as nn

class Dropout:
    # Inspired from https://gist.github.com/nbertagnolli/35eb960d08c566523b4da599f6099b41
    # and https://github.com/pytorch/pytorch/blob/v2.6.0/torch/nn/modules/dropout.py#L35
    def __init__(self, p=0.5):
        if p < 0 or p > 1:
            raise ValueError("p must be a probability in [0, 1].")
        self.p = p
        self.training = True
        self.mask = None

    def forward(self, x):
        """
        Args:
            x (np.ndarray): Input array of any shape.

        Returns:
            np.ndarray: Output array with dropout applied.
        """
        if self.training:
            # Only generate a new mask if one hasn't been set externally.
            if self.mask is None or self.mask.shape != x.shape:
                self.mask = (np.random.rand(*x.shape) >= self.p).astype(x.dtype)
            return (x * self.mask) / (1 - self.p)
        else:
            self.mask = None
            return x

    def backward(self, grad_output):
        if self.training:
            if self.mask is None:
                raise ValueError("Must run forward pass before backward pass in training mode.")
            return (grad_output * self.mask) / (1 - self.p)
        else:
            return grad_output


# ---------- Custom PyTorch Dropout that Stores the Mask ----------
class CustomTorchDropout(nn.Module):
    def __init__(self, p=0.5):
        super(CustomTorchDropout, self).__init__()
        if p < 0 or p > 1:
            raise ValueError("p must be in [0, 1].")
        self.p = p
        self.training = True
        self.mask = None

    def forward(self, x):
        if self.training:
            self.mask = (torch.rand_like(x) >= self.p).float()
            return (x * self.mask) / (1 - self.p)
        else:
            self.mask = None
            return x


# ---------- Instantiate Layers ----------
embed_dim = 192
expansion_factor = 4
p = 0.5
torch_dropout = CustomTorchDropout(p=p)
torch_dropout.training = True

numpy_dropout = Dropout(p=p)
numpy_dropout.training = True

# ---------- Copy mask ----------
np.random.seed(42)
x_np = np.random.randn(embed_dim, embed_dim*expansion_factor).astype(np.float32)
x_torch = torch.tensor(x_np, requires_grad=True)

output_torch = torch_dropout(x_torch)
mask_from_torch = torch_dropout.mask.cpu().numpy().astype(np.float32)
numpy_dropout.mask = mask_from_torch.copy()

# ---------- Forward Pass Comparison ----------
output_torch = torch_dropout(x_torch)
mask_from_torch = torch_dropout.mask.cpu().numpy()
numpy_dropout.mask = mask_from_torch.copy()

output_np = numpy_dropout.forward(x_np)
print("Max absolute difference in forward pass:", np.max(np.abs(output_np - output_torch.detach().cpu().numpy())))

# ---------- Backward Pass Comparison ----------
grad_output_torch = torch.ones_like(x_torch)
output_torch.backward(grad_output_torch)
grad_input_torch = x_torch.grad.detach().cpu().numpy()

grad_output_np = np.ones_like(x_np, dtype=np.float32)
grad_input_np = numpy_dropout.backward(grad_output_np)

print("Max absolute difference in backward pass:", np.max(np.abs(grad_input_np - grad_input_torch)))


Max absolute difference in forward pass: 0.0
Max absolute difference in backward pass: 0.0


### Precision benchmark MLP-Mixer

numpy MLP-Mixer

In [ ]:
class Reduce_np:
    def __init__(self, axis, reduction='mean'):
        self.axis = axis
        self.reduction = reduction

    def forward(self, x):
        self.last_input = x.copy()
        if self.reduction == 'mean':
            return np.mean(x, axis=self.axis)
        else:
            raise NotImplementedError()

    def backward(self, grad_output):
        shape = self.last_input.shape                         # (B, N, dim)
        scale = shape[self.axis]                              # axis = N in MLP_mixer
        grad_input = grad_output / scale
        grad_input = grad_input.reshape(grad_input.shape[0], 1, grad_input.shape[1])  # Reshape to (B, 1, dim) to broadcast along N dim
        return np.broadcast_to(grad_input, shape)             # broadcasts to (B, N, dim)


class PreNormResidual:
    # Inspired from https://github.com/lucidrains/mlp-mixer-pytorch/blob/main/mlp_mixer_pytorch/mlp_mixer_pytorch.py#L7
    def __init__(self, dim, fn):
        self.norm = LayerNorm(dim)
        self.fn = fn
        # fn=TokenMixing/ChannelMixing
        # both return gradients w.r.t input, linear weight & bias, both have cache
    def forward(self, x):
        norm_x, norm_cache = self.norm.forward(x)
        f_out = self.fn.forward(norm_x)
        # Store caches for backward
        self.cache = {'x': x.copy(), 'norm_cache': norm_cache, 'f_cache': self.fn.cache}
        return f_out + x

    def backward(self, d_out):
        # d_out (B, N, dim)
        d_f, f_params_grad = self.fn.backward(d_out)
        # Backprop through norm: use stored norm cache
        d_norm, norm_dw, norm_db = self.norm.backward(d_f, self.cache['norm_cache'])
        self.norm.grad_weight = norm_dw
        self.norm.grad_bias = norm_db
        # The gradient from x is then: d_norm (through norm and f) plus the identity branch d_out.
        d_x = d_norm + d_out

        return d_x, norm_dw, norm_db, f_params_grad


# TokenMixing operates along N dimension
class TokenMixing:
    def __init__(self, num_tokens, expansion_factor, dropout_p):
        inner_dim = int(num_tokens * expansion_factor)
        self.linear1 = Linear(num_tokens, inner_dim)
        self.gelu = GELU()
        self.dropout1 = Dropout(dropout_p)
        self.linear2 = Linear(inner_dim, num_tokens)
        self.dropout2 = Dropout(dropout_p)

    def forward(self, x):
        cache = {}
        cache['x'] = x.copy()
        x_t = np.transpose(x, (0, 2, 1))
        cache['x_t'] = x_t.copy()
        out_l1 = self.linear1.forward(x_t)
        cache['out_l1'] = out_l1.copy()
        out_gelu = self.gelu.forward(out_l1)
        cache['out_gelu'] = out_gelu.copy()
        out_drop1 = self.dropout1.forward(out_gelu)
        cache['out_drop1'] = out_drop1.copy()
        out_l2 = self.linear2.forward(out_drop1)
        cache['out_l2'] = out_l2.copy()
        out_drop2 = self.dropout2.forward(out_l2)
        cache['out_drop2'] = out_drop2.copy()
        out = np.transpose(out_drop2, (0, 2, 1))
        self.cache = cache
        return out

    def backward(self, d_out):
        cache = self.cache
        d_drop2 = np.transpose(d_out, (0, 2, 1))
        d_l2 = self.dropout2.backward(d_drop2)
        d_drop1, grad_w2, grad_b2 = self.linear2.backward(d_l2, cache['out_drop1'])
        d_gelu = self.dropout1.backward(d_drop1)
        d_l1 = self.gelu.backward(d_gelu)
        d_x_t, grad_w1, grad_b1 = self.linear1.backward(d_l1, cache['x_t'])
        d_x = np.transpose(d_x_t, (0, 2, 1))
        self.params_grad = {'linear1': (grad_w1, grad_b1), 'linear2': (grad_w2, grad_b2)}

        # Reset cache to prevent memory leak
        cache = {}
        return d_x, self.params_grad

# ChannelMixing operates along embedding dimension
class ChannelMixing:
    def __init__(self, dim, expansion_factor, dropout_p):
        inner_dim = int(dim * expansion_factor)
        self.linear1 = Linear(dim, inner_dim)
        self.gelu = GELU()
        self.dropout1 = Dropout(dropout_p)
        self.linear2 = Linear(inner_dim, dim)
        self.dropout2 = Dropout(dropout_p)

    def forward(self, x):
        cache = {}
        cache['x'] = x.copy()
        out_l1 = self.linear1.forward(x)
        cache['out_l1'] = out_l1.copy()
        out_gelu = self.gelu.forward(out_l1)
        cache['out_gelu'] = out_gelu.copy()
        out_drop1 = self.dropout1.forward(out_gelu)
        cache['out_drop1'] = out_drop1.copy()
        out_l2 = self.linear2.forward(out_drop1)
        cache['out_l2'] = out_l2.copy()
        out_drop2 = self.dropout2.forward(out_l2)
        cache['out_drop2'] = out_drop2.copy()
        self.cache = cache
        return out_drop2

    def backward(self, d_out):
        cache = self.cache
        d_drop2 = self.dropout2.backward(d_out)
        d_drop1, grad_w2, grad_b2 = self.linear2.backward(d_drop2, cache['out_drop1'])
        d_gelu = self.dropout1.backward(d_drop1)
        d_l1, grad_w1, grad_b1 = self.linear1.backward(self.gelu.backward(d_gelu), cache['x'])
        self.params_grad = {'linear1': (grad_w1, grad_b1), 'linear2': (grad_w2, grad_b2)}

        # Reset cache to prevent memory leak
        cache = {}
        return d_l1, self.params_grad

# ---------- Model ----------
class MLPMixer:
    def __init__(self, num_tokens, input_dim, dim, depth, num_classes,
                 expansion_factor=4, expansion_factor_token=0.5, dropout_p=0.):
        """
        Args:
            num_tokens (int): Number of tokens (N).
            input_dim (int): Input token dimension.
            dim (int): Mixer embedding dimension.
            depth (int): Number of mixer layers.
            num_classes (int): Number of classes for classification.
            expansion_factor (float): Expansion factor for token mixing.
            expansion_factor_token (float): Expansion factor for channel mixing.
            dropout_p (float): Dropout probability.
        """
        self.num_tokens = num_tokens
        self.input_dim = input_dim
        self.dim = dim
        self.depth = depth
        self.num_classes = num_classes
        self.dropout_p = dropout_p
        self.proj = Linear(input_dim, dim) if input_dim != dim else None
        self.mixer_layers = []
        for _ in range(depth):
            token_mixing = PreNormResidual(dim, TokenMixing(num_tokens, expansion_factor, dropout_p))
            channel_mixing = PreNormResidual(dim, ChannelMixing(dim, expansion_factor_token, dropout_p))
            self.mixer_layers.append((token_mixing, channel_mixing))
        self.layer_norm = LayerNorm(dim)
        self.reduce = Reduce_np(axis=1, reduction='mean')
        self.classifier = Linear(dim, num_classes)
        self.cache = {}

    def forward(self, x):
        if self.proj is not None:
            self.cache['proj_in'] = x.copy()
            x = self.proj.forward(x)
        else:
            self.cache['proj_in'] = None
        self.cache['mixer'] = []

        for token_mixing, channel_mixing in self.mixer_layers:
            mixer_cache = {}
            mixer_cache['in'] = x.copy()
            t_out = token_mixing.forward(x)
            mixer_cache['token_cache'] = token_mixing.cache
            c_out = channel_mixing.forward(t_out)
            mixer_cache['channel_cache'] = channel_mixing.cache
            self.cache['mixer'].append(mixer_cache)
            x = c_out

        self.cache['ln_in'] = x.copy()
        x_ln, ln_cache = self.layer_norm.forward(x)
        self.cache['ln_cache'] = ln_cache
        self.cache['reduce_in'] = x_ln.copy()
        x_red = self.reduce.forward(x_ln)
        self.cache['clf_in'] = x_red.copy()
        x_cls = self.classifier.forward(x_red)
        self.cache['output'] = x_cls.copy()
        return x_cls

    def backward(self, grad_output):
        d_clf, _ , _ = self.classifier.backward(grad_output, self.cache['clf_in'])
        d_reduce = self.reduce.backward(d_clf)
        d_ln, _ , _ = self.layer_norm.backward(d_reduce, self.cache['ln_cache'])
        grad = d_ln

        for (token_mixing, channel_mixing), mixer_cache in zip(self.mixer_layers[::-1],
                                                               self.cache['mixer'][::-1]):
            grad, _, _, _ = channel_mixing.backward(grad)
            grad, _, _, _ = token_mixing.backward(grad)

        if self.proj is not None:
            grad, _ , _ = self.proj.backward(grad, self.cache['proj_in'])

        return grad

torch MLPMixer - from lucidrains

In [ ]:
from functools import partial
from einops.layers.torch import Rearrange, Reduce

# Helper: When a non-tuple is provided, treat it as square dimensions.
pair = lambda x: x if isinstance(x, tuple) else (x, x)

class PreNormResidual_torch(nn.Module):
    def __init__(self, dim, fn):
        super().__init__()
        self.fn = fn
        self.norm = nn.LayerNorm(dim)
    def forward(self, x):
        return self.fn(self.norm(x)) + x

def FeedForward_torch(dim, expansion_factor=4, dropout=0., dense=nn.Linear):
    inner_dim = int(dim * expansion_factor)
    return nn.Sequential(
        dense(dim, inner_dim),
        nn.GELU(),
        nn.Dropout(dropout),
        dense(inner_dim, dim),
        nn.Dropout(dropout)
    )

def MLPMixer_torch(num_tokens, input_dim, dim, depth, num_classes,
             expansion_factor=4, expansion_factor_token=0.5, dropout=0.):
    # Inspired from https://github.com/lucidrains/mlp-mixer-pytorch/blob/main/mlp_mixer_pytorch/mlp_mixer_pytorch.py#L26
    # but modify for input shape (B, N, 1) that reflects NIDS dataset
    chan_first, chan_last = partial(nn.Conv1d, kernel_size=1), nn.Linear

    return nn.Sequential(
        nn.Linear(input_dim, dim),
        *[nn.Sequential(
            PreNormResidual_torch(dim, FeedForward_torch(num_tokens, expansion_factor, dropout, dense=chan_first)),
            PreNormResidual_torch(dim, FeedForward_torch(dim, expansion_factor_token, dropout, dense=chan_last))
        ) for _ in range(depth)],
        nn.LayerNorm(dim),
        Reduce('b n c -> b c', reduction='mean'),
        nn.Linear(dim, num_classes)
    )

If you set dropout > 0, the difference will be significant because torch.nn.Dropout initialise a different mask to numpy Dropout. As mention above, there's no way to copy mask
from torch.nn.Dropout so if you want to keep the maximum absolute difference small and using dropout, use CustomTorchDropout

In [ ]:
# ----- Benchmark Setup -----
if __name__ == '__main__':
    num_tokens = 44
    input_dim = 1
    dim = 192
    depth = 8
    num_classes = 2

    # Instantiate the models
    model_torch = MLPMixer_torch(num_tokens, input_dim, dim, depth, num_classes, dropout=0.)
    model = MLPMixer(num_tokens, input_dim, dim, depth, num_classes, dropout_p=0.)

    x_torch = torch.tensor(batch_0, dtype=torch.float32, requires_grad=True)

    # Copy weight and bias from torch.nn.Linear to numpy Linear in MLPMixer
    with torch.no_grad():
        # Copy projection parameters
        torch_proj = model_torch[0]
        if model.proj is not None:
            model.proj.weight[:] = torch_proj.weight.detach().cpu().numpy()
            model.proj.bias[:] = torch_proj.bias.detach().cpu().numpy()

        # For each mixer block, visit token and channel mixing blocks
        for i in range(depth):
            block = model_torch[i+1]  # Each block is an nn.Sequential of length 2.

            # Token Mixing Branch:
            torch_token_branch = block[0]
            feed_seq_token = torch_token_branch.fn
            torch_token_lin1 = feed_seq_token[0]   # nn.Conv1d (inner_dim, num_tokens, 1)
            torch_token_lin2 = feed_seq_token[3]   # nn.Conv1d (num_tokens, inner_dim, 1)
            np_token_branch = model.mixer_layers[i][0].fn
            np_token_branch.linear1.weight[:] = torch_token_lin1.weight.detach().cpu().numpy().squeeze(-1)
            np_token_branch.linear1.bias[:] = torch_token_lin1.bias.detach().cpu().numpy()
            np_token_branch.linear2.weight[:] = torch_token_lin2.weight.detach().cpu().numpy().squeeze(-1)
            np_token_branch.linear2.bias[:] = torch_token_lin2.bias.detach().cpu().numpy()

            # Channel Mixing Branch:
            torch_channel_branch = block[1]
            feed_seq_channel = torch_channel_branch.fn
            torch_channel_lin1 = feed_seq_channel[0]   # nn.Linear(dim, inner_dim)
            torch_channel_lin2 = feed_seq_channel[3]   # nn.Linear(inner_dim, dim)
            np_channel_branch = model.mixer_layers[i][1].fn
            np_channel_branch.linear1.weight[:] = torch_channel_lin1.weight.detach().cpu().numpy()
            np_channel_branch.linear1.bias[:] = torch_channel_lin1.bias.detach().cpu().numpy()
            np_channel_branch.linear2.weight[:] = torch_channel_lin2.weight.detach().cpu().numpy()
            np_channel_branch.linear2.bias[:] = torch_channel_lin2.bias.detach().cpu().numpy()

        # Copy classifier parameters.
        torch_classifier = model_torch[-1]
        model.classifier.weight[:] = torch_classifier.weight.detach().cpu().numpy()
        model.classifier.bias[:] = torch_classifier.bias.detach().cpu().numpy()

    # --- Benchmark forward pass ---
    logits_np = model.forward(batch_0)
    print("logits shape", logits_np.shape)
    logits_torch = model_torch(x_torch)
    print("Max abs difference in forward pass:", np.max(np.abs(logits_np - logits_torch.detach().numpy())))

    # --- Benchmark backward pass ---
    # Create a dummy grad for classifier output
    np.random.seed(42)
    grad_dummy_np = np.random.randn(batch_size, num_classes).astype(np.float32)
    grad_dummy_torch = torch.tensor(grad_dummy_np, dtype=torch.float32)

    # NumPy backward pass: obtain gradient with respect to input.
    grad_input_np = model.backward(grad_dummy_np)
    # PyTorch backward pass:
    if x_torch.grad is not None:
        x_torch.grad.zero_()
    logits_torch.backward(grad_dummy_torch)
    grad_input_torch = x_torch.grad.detach().cpu().numpy()

    bwd_diff = np.max(np.abs(grad_input_np - grad_input_torch))
    print("Max abs difference in backward pass:", bwd_diff)

logits shape (64, 2)
Max abs difference in forward pass: 2.56540936383054e-07
Max abs difference in backward pass: 2.912513954178575e-07


# Precision benchmark for loss, optimiser, optimiser.zero_grad()

### BCELoss with logits

In [ ]:
import math

class BCEWithLogits_np:
    # Inspired from https://github.com/pytorch/pytorch/blob/c64e006fc399d528bb812ae589789d0365f3daf4/aten/src/ATen/native/Loss.cpp#L214-L259
    # binary cross-entropy loss with logits in a numerically stable way using log-sum-exp and backprop it
    def __init__(self, weight=None, pos_weight=None, reduction='mean'):
        """
        Args:
            weight (np.ndarray or None): Optional weight tensor, broadcastable to input.
            pos_weight (np.ndarray or None): Optional weight for positive examples.
            reduction (str): 'mean' or 'sum'
        """
        self.weight = weight
        self.pos_weight = pos_weight
        self.reduction = reduction
        self.cache = {}

    def forward(self, input, target):
        """
        Args:
            input (np.ndarray): logits (B, 2)
            target (np.ndarray): labels (B, 2)
        Returns:
            loss (float or np.ndarray): reduced loss value if reduction is 'mean' or 'sum'; elementwise loss if reduction is 'none'.
        """
        max_val = np.maximum(-input, 0)

        if self.pos_weight is not None:
            log_weight = 1 + (self.pos_weight - 1) * target
            lse = np.log(np.exp(-max_val) + np.exp(-input - max_val))
            loss = (1 - target) * input + log_weight * (lse + max_val)
        else:
            loss = (1 - target) * input + max_val + np.log(np.exp(-max_val) + np.exp(-input - max_val))

        if self.weight is not None:
            loss = loss * self.weight

        self.cache['input'] = input.copy()
        self.cache['target'] = target.copy()

        if self.reduction == 'mean':
            return np.mean(loss)
        elif self.reduction == 'sum':
            return np.sum(loss)

    def backward(self, grad_output=1.0):
        input = self.cache['input']
        target = self.cache['target']

        # Compute sigmoid(input) in a stable way.
        sig = 1 / (1 + np.exp(-input))
        if self.pos_weight is not None:
            grad_input = (sig * (self.pos_weight * target + (1 - target)) - self.pos_weight * target)
        else:
            grad_input = sig - target

        if self.weight is not None:
            grad_input = grad_input * self.weight

        # If reduction is mean, scale the gradient by 1/numel.
        if self.reduction == 'mean':
            grad_input = grad_input * grad_output / input.size
        else:
            grad_input = grad_input * grad_output

        return grad_input

### Adam optimiser

In [ ]:
class ADAM_np:
    # Inspired from https://gist.github.com/aerinkim/dfe3da1000e67aced1c7d9279351cb88
    def __init__(self, params, lr=1e-3, betas=(0.9, 0.99), eps=1e-8, weight_decay=0):
        """
        Args:
            params (list of np.ndarray): List of parameters to optimize. Each parameter must have a field
                `grad` holding the gradient (numpy array of the same shape).
            lr (float): Learning rate.
            betas (tuple): Coefficients for exponential moving averages (beta1, beta2).
            eps (float): Term added to denominator for numerical stability.
            weight_decay (float): L2 penalty.
        """
        self.param_tuples = params
        self.lr = lr
        self.beta1, self.beta2 = betas
        self.eps = eps
        self.weight_decay = weight_decay

        # Create a state for each parameter.
        self.state = {}
        for (p, grad_attr, mod) in self.param_tuples:
            self.state[id(p)] = {
                'step': 0,
                'exp_avg': np.zeros_like(p),
                'exp_avg_sq': np.zeros_like(p)
            }

    def step(self):
        """
        Performs a single optimization step.
        """
        for (p, grad_attr, mod) in self.param_tuples:
            # Get gradient stored in module: e.g., mod.grad_weight or mod.grad_bias.
            grad = getattr(mod, grad_attr)
            state = self.state[id(p)]
            state['step'] += 1

            # Apply weight decay if needed.
            if self.weight_decay != 0:
                grad = grad + self.weight_decay * p

            # Update exponential moving averages.
            state['exp_avg'] = self.beta1 * state['exp_avg'] + (1 - self.beta1) * grad
            state['exp_avg_sq'] = self.beta2 * state['exp_avg_sq'] + (1 - self.beta2) * (grad * grad)

            # Compute the denominator.
            denom = np.sqrt(state['exp_avg_sq']) + self.eps

            # Bias corrections.
            bias_correction1 = 1 / (1 - self.beta1 ** state['step'])
            bias_correction2 = 1 / (1 - self.beta2 ** state['step'])
            adapted_lr = self.lr * bias_correction1 / math.sqrt(bias_correction2)

            # Parameter update.
            p[:] = p - adapted_lr * (state['exp_avg'] / denom)

In [ ]:
np.random.seed(42)
x_torch = torch.tensor(batch_0, dtype=torch.float32, requires_grad=True)
B, N, C = batch_0.shape
num_classes = 2

# Create dummy binary labels (for each sample, a vector of two values in {0,1})
target_np = np.random.randint(0, 2, size=(B, num_classes)).astype(np.float32)
target_torch = torch.tensor(target_np, dtype=torch.float32)

model_torch = MLPMixer_torch(num_tokens, input_dim, dim, depth, num_classes, dropout=0.)
model = MLPMixer(num_tokens, input_dim, dim, depth, num_classes, dropout_p=0.)

# Copy weight and bias from torch.nn.Linear to numpy Linear in MLPMixer
with torch.no_grad():
    # Copy projection parameters
    torch_proj = model_torch[0]
    if model.proj is not None:
        model.proj.weight[:] = torch_proj.weight.detach().cpu().numpy()
        model.proj.bias[:] = torch_proj.bias.detach().cpu().numpy()

    # For each mixer block, visit token and channel mixing blocks
    with torch.no_grad():
        # Copy projection parameters
        torch_proj = model_torch[0]
        if model.proj is not None:
            model.proj.weight[:] = torch_proj.weight.detach().cpu().numpy()
            model.proj.bias[:] = torch_proj.bias.detach().cpu().numpy()

        # For each mixer block, visit token and channel mixing blocks
        for i in range(depth):
            block = model_torch[i+1]  # Each block is an nn.Sequential of length 2.

            # Token Mixing Branch:
            torch_token_branch = block[0]
            feed_seq_token = torch_token_branch.fn
            torch_token_lin1 = feed_seq_token[0]   # nn.Conv1d (inner_dim, num_tokens, 1)
            torch_token_lin2 = feed_seq_token[3]   # nn.Conv1d (num_tokens, inner_dim, 1)
            np_token_branch = model.mixer_layers[i][0].fn
            np_token_branch.linear1.weight[:] = torch_token_lin1.weight.detach().cpu().numpy().squeeze(-1)
            np_token_branch.linear1.bias[:] = torch_token_lin1.bias.detach().cpu().numpy()
            np_token_branch.linear2.weight[:] = torch_token_lin2.weight.detach().cpu().numpy().squeeze(-1)
            np_token_branch.linear2.bias[:] = torch_token_lin2.bias.detach().cpu().numpy()

            # Channel Mixing Branch:
            torch_channel_branch = block[1]
            feed_seq_channel = torch_channel_branch.fn
            torch_channel_lin1 = feed_seq_channel[0]   # nn.Linear(dim, inner_dim)
            torch_channel_lin2 = feed_seq_channel[3]   # nn.Linear(inner_dim, dim)
            np_channel_branch = model.mixer_layers[i][1].fn
            np_channel_branch.linear1.weight[:] = torch_channel_lin1.weight.detach().cpu().numpy()
            np_channel_branch.linear1.bias[:] = torch_channel_lin1.bias.detach().cpu().numpy()
            np_channel_branch.linear2.weight[:] = torch_channel_lin2.weight.detach().cpu().numpy()
            np_channel_branch.linear2.bias[:] = torch_channel_lin2.bias.detach().cpu().numpy()

        # Copy classifier parameters.
        torch_classifier = model_torch[-1]
        model.classifier.weight[:] = torch_classifier.weight.detach().cpu().numpy()
        model.classifier.bias[:] = torch_classifier.bias.detach().cpu().numpy()

# --- Benchmark forward pass ---
logits_np = model.forward(batch_0)
logits_torch = model_torch(x_torch)

In [ ]:
# --- Compute Loss ---
loss_fn_np = BCEWithLogits_np(reduction='mean')
loss_np = loss_fn_np.forward(logits_np, target_np)

loss_fn_torch = nn.BCEWithLogitsLoss(reduction='mean')
loss_torch = loss_fn_torch(logits_torch, target_torch)

print("Max abs difference in forward loss:", np.abs(loss_np - loss_torch.item()))

# --- Backward Pass ---
grad_logits_np = loss_fn_np.backward()
grad_input_np = model.backward(grad_logits_np)

# --- Optimiser Step ---
def get_all_param_tuples(module):
    tuples = []
    # If the module has a weight attribute and it is a NumPy array, add it,
    if hasattr(module, 'weight') and isinstance(module.weight, np.ndarray):
        tuples.append((module.weight, 'grad_weight', module))
    if hasattr(module, 'bias') and module.bias is not None and isinstance(module.bias, np.ndarray):
        tuples.append((module.bias, 'grad_bias', module))
    # Also check for a submodule 'norm'
    if hasattr(module, 'norm'):
        norm_mod = module.norm
        if hasattr(norm_mod, 'weight') and isinstance(norm_mod.weight, np.ndarray):
            tuples.append((norm_mod.weight, 'grad_weight', norm_mod))
        if hasattr(norm_mod, 'bias') and norm_mod.bias is not None and isinstance(norm_mod.bias, np.ndarray):
            tuples.append((norm_mod.bias, 'grad_bias', norm_mod))
    # Recursively traverse the module's __dict__, but skip keys known to be caches.
    for key, value in module.__dict__.items():
        if hasattr(value, 'forward') and callable(value.forward) and value is not module:
            tuples.extend(get_all_param_tuples(value))
        elif isinstance(value, (list, tuple)):
            for item in value:
                if hasattr(item, 'forward') and callable(item.forward):
                    tuples.extend(get_all_param_tuples(item))
    return tuples

numpy_params = get_all_param_tuples(model)
optimiser_np = ADAM_np(numpy_params, lr=0.06, weight_decay=0.1)
optimiser_np.step()

optimizer_torch = torch.optim.Adam(model_torch.parameters(), lr=0.06, weight_decay=0.1)
optimizer_torch.zero_grad()
loss_torch.backward()
optimizer_torch.step()

Max abs difference in forward loss: 5.7054438284964704e-08


KeyError: 'clf_in'

In [ ]:
# --- Compare classifier weight and bias after update ---
classifier_weight_np = model.classifier.weight.copy()
classifier_bias_np = model.classifier.bias.copy()

classifier_weight_torch = model_torch[-1].weight.detach().cpu().numpy()
classifier_bias_torch = model_torch[-1].bias.detach().cpu().numpy()

weight_diff = np.max(np.abs(classifier_weight_np - classifier_weight_torch))
bias_diff = np.max(np.abs(classifier_bias_np - classifier_bias_torch))

print("Max abs difference in classifier weight after update:", weight_diff)
print("Max abs difference in classifier bias after update:", bias_diff)

Max abs difference in classifier weight after update: 0.00013391674
Max abs difference in classifier bias after update: 3.427267e-07


In [ ]:
np_layernorm_weight = model.layer_norm.weight.copy()
np_layernorm_bias   = model.layer_norm.bias.copy()

torch_layernorm = None
for module in model_torch.modules():
    # This will return every submodule of your Torch model.
    if isinstance(module, nn.LayerNorm):
        torch_layernorm = module
        break

if torch_layernorm is None:
    raise RuntimeError("Could not find a LayerNorm module in your Torch model.")

torch_layernorm_weight = torch_layernorm.weight.detach().cpu().numpy()
torch_layernorm_bias   = torch_layernorm.bias.detach().cpu().numpy()

print("Max abs difference in LayerNorm weight:", np.max(np.abs(np_layernorm_weight - torch_layernorm_weight)))
print("Max abs difference in LayerNorm bias:", np.max(np.abs(np_layernorm_bias - torch_layernorm_bias)))

Max abs difference in LayerNorm weight: 5.9604645e-08
Max abs difference in LayerNorm bias: 0.11999719


In [ ]:
np_proj_weight = model.proj.weight.copy()
np_proj_bias   = model.proj.bias.copy()

torch_proj_weight = model_torch[0].weight.detach().cpu().numpy()
torch_proj_bias = model_torch[0].bias.detach().cpu().numpy()

print("Max abs difference in projection weight:", np.max(np.abs(np_proj_weight - torch_proj_weight)))
print("Max abs difference in projection bias:", np.max(np.abs(np_proj_bias - torch_proj_bias)))

Max abs difference in projection weight: 6.7166984e-06
Max abs difference in projection bias: 1.0665506e-05


In [ ]:
import numpy as np

a = np.array([np.array([1, 2, 3]),
              np.array([4, 5]),
              np.array([6, 7, 8, 9])])
print("Array a:", a)
print("a.shape:", a.shape)  # This will typically be (3,) because it is an object array.

# Now, try to create an array of zeros with the same shape as a.
b = np.zeros_like(a)

print("a:", a)
print("dtype:", a.dtype)

# Attempt to create a zeros array with the same shape as a.
b = np.zeros_like(a)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (3,) + inhomogeneous part.

In [ ]:
import numpy as np

x = np.array([[1, 2], [3, 4]])
x[...] = 0
print(x)

[[0 0]
 [0 0]]


### Optimizer.zero_grad()

In [ ]:
def clear_auxiliary(module):
    """
    Recursively clears auxiliary attributes from the module and its submodules.
    This includes attributes with keys 'cache', 'ctx', 'params_grad', and 'mask'.

    For each key:
      - If its value is a dict, it is set to {}.
      - Otherwise, it is set to None.
    """
    aux_keys = ('cache', 'ctx', 'params_grad', 'mask')
    # Clear any auxiliary attribute on this module.
    for key in aux_keys:
        if key in module.__dict__:
            current = module.__dict__[key]
            if isinstance(current, dict):
                setattr(module, key, {})
            else:
                setattr(module, key, None)

    # Now recursively traverse submodules.
    for key, value in module.__dict__.items():
        # Skip aux keys already handled.
        if key in aux_keys:
            continue

        # If the attribute is a list or tuple, iterate through its items.
        if isinstance(value, (list, tuple)):
            for item in value:
                if isinstance(item, tuple):
                    for subitem in item:
                        if hasattr(subitem, 'forward') and callable(subitem.forward):
                            clear_auxiliary(subitem)
                elif hasattr(value, 'forward') and callable(value.forward):
                    # Unlikely: this checks if the entire list itself has a forward().
                    clear_auxiliary(value)
                elif hasattr(item, 'forward') and callable(item.forward):
                    clear_auxiliary(item)
        # If the attribute is a submodule (has callable forward), recurse.
        elif hasattr(value, 'forward') and callable(value.forward) and value is not module:
            clear_auxiliary(value)

# Example usage:
# After an optimizer step or at the start/end of an iteration, call:
clear_auxiliary(model)

In [ ]:
def print_dict(d, indent=0):
    prefix = " " * indent
    if not isinstance(d, dict):
        print(f"{prefix}{d}")
        return
    for key, value in d.items():
        if isinstance(value, np.ndarray):
            print(f"{prefix}{key}: array of shape {value.shape}")
        elif isinstance(value, dict):
            print(f"{prefix}{key}: dict:")
            print_dict(value, indent+4)
        elif isinstance(value, (int, float, str)):
            print(f"{prefix}{key}: {value}")
        else:
            # If it is a list, tuple, or other object, show the type
            print(f"{prefix}{key}: {type(value).__name__}")

def print_model_details(module, indent=0):
    prefix = " " * indent
    # Print current module class name.
    print(f"{prefix}{module.__class__.__name__} details:")
    for key, value in module.__dict__.items():
        # If key is one of the ones we want to inspect its content:
        if key in ('ctx', 'cache', 'params_grad'):
            print(f"{prefix}  {key}:")
            print_dict(value, indent+4)
        elif isinstance(value, np.ndarray):
            print(f"{prefix}  {key}: array of shape {value.shape}")
        else:
            if isinstance(value, (int, float, str)):
                print(f"{prefix}  {key}: {value}")
            else:
                print(f"{prefix}  {key}: {type(value).__name__}")

    # Recursively descend for each attribute.
    for key, value in module.__dict__.items():
        # If the attribute is a list or tuple, iterate through its items.
        if isinstance(value, (list, tuple)):
            for idx, item in enumerate(value):
                if isinstance(item, tuple):
                    for subitem in item:
                        if hasattr(subitem, 'forward') and callable(subitem.forward):
                            print_model_details(subitem, indent+4)
                elif hasattr(value, 'forward') and callable(value.forward):
                    print_model_details(value, indent+4)
                elif hasattr(item, 'forward') and callable(item.forward):
                    print_model_details(item, indent+4)
        # If the attribute is a module (has callable forward), recurse.
        elif hasattr(value, 'forward') and callable(value.forward) and value is not module:
            print_model_details(value, indent+4)

print_model_details(model)

MLPMixer details:
  num_tokens: 44
  input_dim: 1
  dim: 192
  depth: 8
  num_classes: 2
  dropout_p: 0.0
  proj: Linear
  mixer_layers: list
  layer_norm: LayerNorm
  reduce: Reduce_np
  classifier: Linear
  cache:
    Linear details:
      in_features: 1
      out_features: 192
      weight: array of shape (192, 1)
      bias: array of shape (192,)
      grad_weight: array of shape (192, 1)
      grad_bias: array of shape (192,)
    PreNormResidual details:
      norm: LayerNorm
      fn: TokenMixing
      cache:
        LayerNorm details:
          dim: 192
          weight: array of shape (192,)
          bias: array of shape (192,)
          grad_weight: array of shape (192,)
          grad_bias: array of shape (192,)
        TokenMixing details:
          linear1: Linear
          gelu: GELU
          dropout1: Dropout
          linear2: Linear
          dropout2: Dropout
          cache:
          params_grad:
            Linear details:
              in_features: 44
           